In [1]:
import sys
sys.path.append("../src/") 
import os
import model_manipulation as mm

import cobra
import libsbml
from cobra import Model, Reaction, Metabolite, manipulation, test
import pandas as pd
import csv
import copy

In [2]:
ios2164_orig = cobra.io.read_sbml_model("../model/ios2164_orig.xml")

ios2164_orig.solver = 'gurobi'

#Set all non-zero flux bounds to 1e-6

mm.set_flux_to_1e6(ios2164_orig)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-12


In [3]:
#Change PEPC stoichiometry from consuming CO2 to consuming HCO3
co2_c = ios2164_orig.metabolites.get_by_id('co2_c')
hco3_c = ios2164_orig.metabolites.get_by_id('hco3_c')
h_c = ios2164_orig.metabolites.get_by_id('h_c')
h2o_c = ios2164_orig.metabolites.get_by_id('h2o_c')

PEPC_old = ios2164_orig.reactions.PPCc
PEPC_temp = PEPC_old

# # #Removes old co2, h and h2o in favor of hco3
PEPC_temp.add_metabolites({hco3_c:1, co2_c:-1})

ios2164_orig.reactions.PPCc = PEPC_temp

In [ ]:
ios2164_orig.reactions.PPCc

In [ ]:
#let's add a reaction specific for phloem, which includes only amino acids, starch and sucrose
#Values are from Poolman et al. (2014) and is rescaled from mol/gcw * time to mmol/gcw * time (similar to ios2164 and Arnold's Atmodel)

#Read from csv
read_phl = pd.read_csv('../misc/phloem_sap_comp.csv')
phl_dict = dict(read_phl.values)


model_rxn_list = list()
for items in phl_dict:
    #Update reaction signs first so it's all negative 
    phl_dict[items] = phl_dict[items] * -1
    
    #Retrieve pointer from original model to retrieve metabolite
    model_rxn = ios2164_orig.metabolites.get_by_id(items)
    model_rxn_list.append(model_rxn)

#Update rxns to pointers to orig model
for rxns in model_rxn_list:
    phl_dict[rxns] = phl_dict.pop(rxns.id)

#Generate new phloem reaction
phl_rxn = Reaction('DM_Phloem')
phl_rxn.name = 'export reaction for phloem sap'
phl_rxn.add_metabolites(phl_dict)

ios2164_orig.add_reaction(phl_rxn)
    

In [ ]:
#Add NGAM reactions based on B&B's measurements
#This is a  good way to account for metabolic costs. However, the only limitation would this be would be that the reaction bounds are constant and non-variable

ngam_atp_c = Reaction('ngam_atp_c')
ngam_atp_s = Reaction('ngam_atp_s')
ngam_atp_x = Reaction('ngam_atp_x')
ngam_atp_m = Reaction('ngam_atp_m')

#Values from B&B(2019) are the ff (Units are in umol atp m-2 s-1)
# cytosol	0.0427
# chloroplast	0.1527
# mitochondria	0.0091
# peroxisome	0.0076

ngam_atp_c.name = 'ATP NGAM for Cytosol'
ngam_atp_s.name = 'ATP NGAM for Chloroplast (Plastid)'
ngam_atp_x.name = 'ATP NGAM for Peroxisome'
ngam_atp_m.name = 'ATP NGAM for Mitochondria'

atp_c = ios2164_orig.metabolites.atp_c
adp_c = ios2164_orig.metabolites.adp_c
pi_c = ios2164_orig.metabolites.pi_c
h_c = ios2164_orig.metabolites.h_c

atp_s = ios2164_orig.metabolites.atp_s
adp_s = ios2164_orig.metabolites.adp_s
pi_s = ios2164_orig.metabolites.pi_s
h_s = ios2164_orig.metabolites.h_s

atp_x = ios2164_orig.metabolites.atp_x
adp_x = ios2164_orig.metabolites.adp_x
pi_x = ios2164_orig.metabolites.pi_x
h_x =  ios2164_orig.metabolites.h_x

atp_m = ios2164_orig.metabolites.atp_m
adp_m = ios2164_orig.metabolites.adp_m
pi_m = ios2164_orig.metabolites.pi_m
h_m = ios2164_orig.metabolites.h_m



ngam_atp_c.add_metabolites({atp_c:-1, adp_c:1, pi_c:1, h_c: 1})
ngam_atp_s.add_metabolites({atp_s:-1, adp_s:1, pi_s:1, h_s: 1})
ngam_atp_x.add_metabolites({atp_x:-1, adp_x:1, pi_x:1, h_x: 1})
ngam_atp_m.add_metabolites({atp_m:-1, adp_m:1, pi_m:1, h_m: 1})

#Fix bounds of NGAM reaction. Values are from B&B. 
#However, what I'll implement instead will be Topfer's linear NGAM approach, which scales NGAM with PPFD



ios2164_orig.add_reaction(ngam_atp_c)
ios2164_orig.add_reaction(ngam_atp_s)
ios2164_orig.add_reaction(ngam_atp_x)
ios2164_orig.add_reaction(ngam_atp_m)



In [ ]:
#Change bounds of NGAM reaction, turn it to 0 in the meantime

ngam_atp_c.bounds = (0,0) #0.0427
ngam_atp_s.bounds = (0,0) #0.1527
ngam_atp_x.bounds = (0,0) #0.0076
ngam_atp_m.bounds = (0,0) #0.0091

In [ ]:
#NGAM for NADPH should be defined in the following compartments:

#cytosol, mitochondria and plastid

ngam_nadphox_c = Reaction('ngam_nadphox_c')
ngam_nadphox_s = Reaction('ngam_nadphox_s')
ngam_nadphox_m = Reaction('ngam_nadphox_m')

ngam_nadphox_c.name = "Generic cytosolic NADPH Oxidase for NGAM"
ngam_nadphox_s.name = "Generic stromal NADPH Oxidase for NGAM"
ngam_nadphox_m.name = "Generic mitochondrial NADPH Oxidase for NGAM"

#reaction would be the following
# #NADPH + 2O2 ↔ NADP+ + 2O2− + H+
#From Cheung et al (2013)'s work on modelling the effects of NGAM on plant fluxes, which in turn was  followed 
# NADP_c + x_NADPHWorkc + WATER_c ⇔ NADPH_c + ×met_2259 + PROTON_c	
#Met_2259 is oxygen

#Get all the constituent metabolites for the generic NADPH oxidase reaction in each compartment
nadph_c = ios2164_orig.metabolites.nadph_c
nadph_s = ios2164_orig.metabolites.nadph_s
nadph_m = ios2164_orig.metabolites.nadph_m


nadp_c = ios2164_orig.metabolites.nadp_c
nadp_s = ios2164_orig.metabolites.nadp_s
nadp_m = ios2164_orig.metabolites.nadp_m

h_c = ios2164_orig.metabolites.h_c
h_s = ios2164_orig.metabolites.h_s
h_m = ios2164_orig.metabolites.h_m

o2_c = ios2164_orig.metabolites.o2_c
o2_s = ios2164_orig.metabolites.o2_s
o2_m = ios2164_orig.metabolites.o2_m

h2o_c = ios2164_orig.metabolites.h2o_c
h2o_s =  ios2164_orig.metabolites.h2o_s
h2o_m =  ios2164_orig.metabolites.h2o_m


ngam_nadphox_c.add_metabolites({nadph_c:-1, o2_c:-1, h_c:-1, nadp_c:1, h2o_c: 1})
ngam_nadphox_s.add_metabolites({nadph_s:-1, o2_s:-1, h_s:-1, nadp_s:1, h2o_s: 1})
ngam_nadphox_m.add_metabolites({nadph_m:-1, o2_m:-1, h_m:-1, nadp_m:1, h2o_m: 1})

ngam_nadphox_c.bounds=(0, 1000)
ngam_nadphox_s.bounds=(0,1000)
ngam_nadphox_m.bounds=(0,1000)

ios2164_orig.add_reaction(ngam_nadphox_c)
ios2164_orig.add_reaction(ngam_nadphox_s)
ios2164_orig.add_reaction(ngam_nadphox_m)



In [ ]:



#Add constraints such that the NADPH oxidase ratios are equally distributed in each compartment
nadphox_c_s = mm.set_fix_flux_ratio({ngam_nadphox_c.id:1, ngam_nadphox_s.id:1},ios2164_orig)
nadphox_c_s.name = "nadphox_cs_ratio"
nadphox_s_m = mm.set_fix_flux_ratio({ngam_nadphox_s.id:1, ngam_nadphox_m.id:1}, ios2164_orig)
nadphox_s_m.name = "nadphox_sm_ratio"

ios2164_orig.add_cons_vars(nadphox_c_s)
ios2164_orig.add_cons_vars(nadphox_s_m)



In [ ]:

#Add constraint such that ATP:NADPH oxidase ratio is at 3:1

fex_nadphoxc =  mm.get_flux_exp(ios2164_orig, ngam_nadphox_c)
fex_nadphoxs =  mm.get_flux_exp(ios2164_orig, ngam_nadphox_s)
fex_nadphoxm =  mm.get_flux_exp(ios2164_orig, ngam_nadphox_m)
fex_atpc = mm.get_flux_exp(ios2164_orig, ngam_atp_c)
fex_atps =  mm.get_flux_exp(ios2164_orig, ngam_atp_s)
fex_atpm = mm.get_flux_exp(ios2164_orig, ngam_atp_m)
fex_atpx =mm.get_flux_exp(ios2164_orig, ngam_atp_x)


nadphox_atpase = ios2164_orig.problem.Constraint(3*(fex_nadphoxc+fex_nadphoxs +fex_nadphoxm) 
                                     - 1*(fex_atpc + fex_atps + fex_atpm),
                                     lb=0,ub=0)
nadphox_atpase.name = "nadphox_atpase_ratio"


ios2164_orig.add_cons_vars(nadphox_atpase)


In [ ]:
ios2164_orig.constraints.nadphox_cs_ratio.expression

In [ ]:
#Restrict all reactions to white light only
for lights in ios2164_orig.reactions:
    if 'PRISM' in lights.id:
        ios2164_orig.reactions.get_by_id(lights.id).bounds = (-1e6,1e6)
        #Constrain all non-white light reactions to 0
        if 'white' not in lights.id:
            ios2164_orig.reactions.get_by_id(lights.id).bounds = (0,0)


In [ ]:
#Add Rubisco Oxygenase-specific reaction to model

#First retrieve the carboxylase reaction
rbc_rxn = ios2164_orig.reactions.RBPCs


#Define reaction. Id is RBPOs
rbo_rxn = Reaction('RBPOs')
rbo_rxn.name = 'Ribulose-biphosphate Oxygenase'

#Components of oxygenase reaction are as follows
#o2[s] + h2o[s] + rb15bp[s] -> 3pg[s] + 2pglyc[s]
#Interestingly, Arnold's model accounts for the generation of protons by the rubisco reaction.
#There's also support from literature on how protonation may also enhance CO2 capture by a plant Cell.
#To accomodate this I'll just add the reaction specifically for both RBO and RBC reactions.

#Retrieve metabolites
o2_s = ios2164_orig.metabolites.o2_s
h2o_s = ios2164_orig.metabolites.h2o_s
rb15bp_s = ios2164_orig.metabolites.rb15bp_s
h_s = ios2164_orig.metabolites.h_s
_3pg_s = ios2164_orig.metabolites.get_by_id('3pg_s')
_2pglyc_s = ios2164_orig.metabolites.get_by_id('2pglyc_s')

#Define Stoichiometry
rbo_rxn.add_metabolites({o2_s:-1.0, h2o_s:-1.0, rb15bp_s:-1, h_s:1, _3pg_s: 1, _2pglyc_s: 1})

#Copy other details from rbc_rxn
rbo_rxn.subsystem = rbc_rxn.subsystem
rbo_rxn.bounds = rbc_rxn.bounds
#Define encoded genes; copied from ios2164 supplementary
rbo_rxn.gene_reaction_rule = rbc_rxn.gene_name_reaction_rule

#Add to model
ios2164_orig.add_reaction(rbo_rxn)

ios2164_orig.repair

ios2164_orig.reactions.RBPOs
        
#addtl refs:
#Long, B. M., Förster, B., Pulsford, S. B., Price, G. D., & Badger, M. R. (2021). 
#Rubisco proton production can drive the elevation of CO2 within condensates and carboxysomes. 
#Proceedings of the National Academy of Sciences, 118(18).

In [ ]:
# #Add proton production to original carboxylase reaction
# #From B&B (2019), Long et al. (2021)
# rbc_rxn.add_metabolites({h_s:-12.0})
rbc_rxn

In [ ]:
#Let's add a specific demand reaction for oxygen

dm_o2 = Reaction('DM_o2(c)')

dm_o2.name = "Demand reaction for oxygen"

o2_c = ios2164_orig.metabolites.o2_c


dm_o2.add_metabolites({o2_c:-1.0})

dm_o2.bounds = (0, 1000)

ios2164_orig.add_reaction(dm_o2)

ios2164_orig.repair()

    

In [ ]:
ios2164_m = copy.deepcopy(ios2164_orig)
for met in ios2164_m.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '0'
        met.compartment = str(met.compartment) + '0'

    
        
for rxn in ios2164_m.reactions:
    if "EX_" not in rxn.id: #Exclude exchange reactions
        rxn.id = str(rxn.id)  + '_M'


#Change name of reaction 
ios2164_m.repair()


#Change name of NGAM constraints
ios2164_m.constraints.nadphox_atpase_ratio.name = str(ios2164_m.constraints.nadphox_atpase_ratio.name) + "_M"
ios2164_m.constraints.get("nadphox_cs_ratio").name = str(ios2164_m.constraints.get("nadphox_cs_ratio").name) +"_M"
ios2164_m.constraints.get("nadphox_sm_ratio").name = str(ios2164_m.constraints.get("nadphox_sm_ratio").name) +"_M"

In [ ]:
ios2164_bs = copy.deepcopy(ios2164_orig)

for met in ios2164_bs.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '1'
        met.compartment = str(met.compartment) + '1'
    
for rxn in ios2164_bs.reactions:
    if "EX_" not in rxn.id: #Exclude exchange reactions
        rxn.id = str(rxn.id)  + '_BS'
#     if "EX_" in rxn.id: #Rename media reactions
#         rxn.name = "BS " + str(rxn.name)




#Replace constraint names
ios2164_bs.constraints.nadphox_atpase_ratio.name = str(ios2164_bs.constraints.nadphox_atpase_ratio.name) + "_BS"
ios2164_bs.constraints.get("nadphox_cs_ratio").name = str(ios2164_bs.constraints.get("nadphox_cs_ratio").name) +"_BS"
ios2164_bs.constraints.get("nadphox_sm_ratio").name = str(ios2164_bs.constraints.get("nadphox_sm_ratio").name) +"_BS"

In [ ]:
ios2164_2cell = ios2164_m.merge(ios2164_bs, inplace=True, objective = 'sum')


In [ ]:
#Turn off all NGAMs in the meantime. I can return it when I perform the  simulations directly
for rxns in ios2164_orig.reactions:
    
    if 'ngam_atp' in rxns.id:
        ios2164_orig.reactions.get_by_id(rxns.id).bounds = (0,0)
        print(rxns.id, rxns.bounds)

In [ ]:
#Apparently compartments don't update after  merging so I've updated it na lang after that

ios2164_2cell.compartments = {
 'c0': 'M Cell Cytoplasm',
 's0': 'M Cell Plastid',
 'r0': 'M Cell Endoplasmic_reticulum',
 'm0': 'M Cell Mitochondrion',
 'e': 'Extracellular',
 'u0': 'M Cell Thylakoid',
 'x0': 'M Cell Peroxisome',
 'v0': 'M Cell Vacuole',
 'c1': 'BS Cell Cytoplasm',
 's1': 'BS Cell Plastid',
 'r1': 'BS Cell Endoplasmic_reticulum',
 'm1': 'BS Cell Mitochondrion',
 'u1': 'BS Cell Thylakoid',
 'x1': 'BS Cell Peroxisome',
 'v1': 'BS Cell Vacuole'}

#Also need to rebuild the model medium
ios2164_2cell.repair()


From Blatke & Brautigam (2019):

"The mesophyll and bundle sheath networks are connected by a range of cytosolic transport metabolites including amino acids, sugars (glucose, fructose, sucrose, trehalose, ribose), single phosphorylated sugar (glucose-6-phosphate, glucose-1-phosphate, fructose-6-phosphate, sucrose-6-phosphate), mono-/di-/tri-carboxylic acids (phosphoenolpyruvate, pyruvate, citrate, cis-aconitate, isocitrate, α-ketoglutarate, succinate, fumarate, malate), glyceric acids (2-Phosphoglycerate, 3-Phosphoglycerate), glycolate, glycerate, glyceraldehyde-3-phosphate, di-hydroxyacetone-phosphate and CO2. 

Nucleotides, NAD/NADH, NADP/NADPH, pyrophosphate, inorganic phosphate are not considered as transport metabolites. 

Oxaloacetate has been excluded as transport metabolite since concentrations of oxaloacetate are very low in vivo and it is reasonably unstable in aqueous solutions. 

Other small molecules that can be imported by the bundle sheath from the environment, as well as protons and HCO3-, are not exchanged between the two cell types.

#So bale yung mga organic na soluble lang with reasonably long cellular half-lives.

In [ ]:
#Open the initial curated list to get the metabolite ids
transport_rxns = pd.read_csv('../model/initial_list_transport_rxns.csv')
#Ok it gets the reactions now

for item in ios2164_orig.metabolites:
    if item.compartment == "c":
        if item.id in transport_rxns.id.values:
            pd_rxn = Reaction()
            #Generate reaction name
            pd_rxn.id = str(item.id[:-2] + '_pd') #Replaces suffix with '_pd'
            pd_rxn.name = str('Plasmodesmatal transport of ' + item.name)
            pd_rxn.subsystem = str('PD Transport Reaction')
            #Reaction is reversible
            pd_rxn.lower_bound = -1000
            pd_rxn.upper_bound = 1000
            
            
            #Generate metabolite names
            
            
            met_mcell = str(item.id) + '0'
            met_bcell = str(item.id) + '1'
            
            met_mcell = ios2164_2cell.metabolites.get_by_id(met_mcell)
            met_bcell = ios2164_2cell.metabolites.get_by_id(met_bcell)
            pd_rxn.add_metabolites({met_mcell:-1.0, met_bcell:1.0})
            ios2164_2cell.add_reaction(pd_rxn)
            
ios2164_2cell.repair()


#Print how many reactions added to model
printout = 'added ' + str(len(transport_rxns)) + ' PD reactions to model'
print(printout)


In [ ]:
for pds in ios2164_2cell.reactions:
    if 'pd' in pds.id:
        print(pds.id)

In [ ]:
#Set all fluxes to 0 muna to make it all easy.
for rxns in ios2164_2cell.exchanges:
    rxn_id = rxns.id
    ios2164_2cell.reactions.get_by_id(rxn_id).bounds = (0, 0)


In [ ]:

#Set Sucrose PD to be non-reversible (M -> BS) so as to properly model bulk flow
ios2164_2cell.reactions.get_by_id('sucr_pd').bounds = (0, 1000)



In [ ]:

mm.set_flux_to_1e6(ios2164_2cell)

for item in ios2164_2cell.reactions:
    print(item.id, item.bounds)

In [ ]:
# Change objective function so as to reflect the two cell state


ios2164_2cell.objective = {ios2164_2cell.reactions.Straw_Biomass_M: 1,
                  ios2164_2cell.reactions.Straw_Biomass_BS: 1}


print(ios2164_2cell.objective)


In [ ]:
for items in ios2164_2cell.reactions:
    print(items.id, items.bounds)

In [ ]:
mets = open('../misc/metabolites.csv', 'w')
rxns = open('../misc/reactions.csv', 'w')
cons = open('../misc/constraints.csv', 'w')

met_write = csv.writer(mets)
rxns_write = csv.writer(rxns)
cons_write = csv.writer(cons)

for item in ios2164_2cell.metabolites:
    row = [item.id, item.name]
    met_write.writerow(row)
    

for item in ios2164_2cell.reactions:
    row = [item.id, item.name, item.build_reaction_string(), item.bounds]
    rxns_write.writerow(row)

for item  in ios2164_2cell.constraints:
    row =  [str(item.name), str(item.expression)]
    cons_write.writerow(row)
    
    


In [ ]:
cobra.io.write_sbml_model(ios2164_2cell, filename= "../model/ios2164_2cell.xml")
#Save Json for mapping
cobra.io.save_json_model(ios2164_2cell, filename="../model/ios2164_2cell.json")
#Save one-cell model to incorporate updates
cobra.io.write_sbml_model(ios2164_orig, filename="../model/ios2164_1cell.xml")